In [1]:
import time
import tensorflow as tf
import psutil

class PowerMonitor:
    def __init__(self):
        self.gpu_available = tf.config.list_physical_devices('GPU')
        
        # Hardware power specifications (adjust these values for your system)
        self.cpu_tdp = 65    # Typical TDP for desktop CPUs in watts
        self.gpu_tdp = 250   # Typical TDP for desktop GPUs in watts
        
    def get_stats(self):
        """Get system stats with power estimation"""
        stats = {
            'timestamp': time.time(),
            'cpu_%': psutil.cpu_percent(interval=0.1),
            'ram_mb': psutil.virtual_memory().used / (1024**2),
            'gpu_mem_mb': 0,
            'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85  # Base CPU power
        }
        
        if self.gpu_available:
            try:
                # TensorFlow GPU memory monitoring
                mem_info = tf.config.experimental.get_memory_info('GPU:0')
                stats.update({
                    'gpu_mem_mb': mem_info['current'] / (1024**2),
                    'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85 + 
                              self.gpu_tdp * 0.5 * 0.75  # Add GPU power estimate
                })
            except:
                pass
                
        return stats

# Initialize monitor
monitor = PowerMonitor()

# Celeb

In [3]:

import h5py

# Open the HDF5 file for reading
#with h5py.File("D:\\thesis\dataset\deepfake dataset\images_celeb_224R_processed.h5", "r") as h5f:
with h5py.File("D:\\thesis\dataset\Celeb-Df-v2\images_celeb_balanced_224R_processed.h5", "r") as h5f:
    # Load HOG features
    real_frames_array1 = h5f["ori_actor"][:]
    fake_frames_array1 = h5f["ori_youtube"][:]
import numpy as np
import cv2

# Function to resize images from (224, 224) to (160, 160)
def resize_images(image_array, target_size=(160, 160)):
    resized_images = np.array([cv2.resize(img, target_size) for img in image_array])
    return resized_images


# Resize the images
real_frames_array1 = resize_images(real_frames_array1, target_size=(160, 160))
fake_frames_array1 = resize_images(fake_frames_array1, target_size=(160, 160))

# Checking the new shapes
real_frames_array1.shape, fake_frames_array1.shape


((9870, 160, 160, 3), (11274, 160, 160, 3))

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


# Split the data into train (70%) and test (30%)
X_train_real, X_test_real = train_test_split(real_frames_array1, test_size=0.3, random_state=42)
X_train_fake, X_test_fake = train_test_split(fake_frames_array1, test_size=0.3, random_state=42)


# Output the shape to confirm the split dimensions
print("Training data real shape:", X_train_real.shape)
print("Testing real data shape:", X_test_real.shape)
print("Training fake data shape:", X_train_fake.shape)
print("Testing  fake data shape:", X_test_fake.shape)
########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
train_hog_real, val_hog_real = extract_validation(X_train_real)
train_hog_fake, val_hog_fake = extract_validation(X_train_fake)

# Print results for verification
print(f"train_hog_real: {len(train_hog_real)} images, val_hog_real: {len(val_hog_real)} images")
print(f"train_hog_fake: {len(train_hog_fake)} images, val_hog_fake: {len(val_hog_fake)} images")


############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


train_labels_real = np.zeros(len(train_hog_real), dtype=int)
train_labels_fake = np.ones(len(train_hog_fake), dtype=int)

# Concatenate all training datasets into a single `train` variable
train = np.concatenate([train_hog_real, train_hog_fake], axis=0)
train_labels=np.concatenate([train_labels_real, train_labels_fake], axis=0)



test_labels_real = np.zeros(len(X_test_real), dtype=int)
test_labels_fake = np.ones(len(X_train_fake), dtype=int)

# Concatenate all testing datasets into a single `test` variable
test = np.concatenate([X_test_real, X_train_fake], axis=0)
test_labels = np.concatenate([test_labels_real, test_labels_fake], axis=0)



val_labels_real = np.zeros(len(val_hog_real), dtype=int)
val_labels_fake = np.ones(len(val_hog_fake), dtype=int)

# Concatenate all validation datasets into a single `val` variable
val = np.concatenate([val_hog_real, val_hog_fake], axis=0)
val_labels = np.concatenate([val_labels_real, val_labels_fake], axis=0)

# Print the results for verification
print(f"Total train: {len(train)} images")
print(f"Total test: {len(test)} images")
print(f"Total val: {len(val)} images")


# Print results for verification
print(f"Train Labels: {len(train_labels)} ")
print(f"Test Labels: {len(test_labels)} ")
print(f"Val Labels: {len(val_labels)} ")



Training data real shape: (6909, 160, 160, 3)
Testing real data shape: (2961, 160, 160, 3)
Training fake data shape: (7891, 160, 160, 3)
Testing  fake data shape: (3383, 160, 160, 3)
train_hog_real: 6218 images, val_hog_real: 691 images
train_hog_fake: 7101 images, val_hog_fake: 790 images
Total train: 13319 images
Total test: 10852 images
Total val: 1481 images
Train Labels: 13319 
Test Labels: 10852 
Val Labels: 1481 


In [10]:
print("=== DATA LOADING ===")
start = monitor.get_stats()

=== DATA LOADING ===


In [3]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50

def build_resnet_model(input_shape=(160, 160, 3)):
    """
    Creates a ResNet50-based deepfake detection model
    
    Args:
        input_shape: Tuple specifying input dimensions (height, width, channels)
    
    Returns:
        Compiled Keras model
    """
    # Load pre-trained ResNet50 without top classification layer
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze the base model
    base_model.trainable = False
    
    # Create new model on top
    inputs = layers.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)  # Normalization
    
    # ResNet preprocessing (matches ImageNet normalization)
    x = layers.Lambda(lambda img: (img * 255) - 
                      tf.constant([103.939, 116.779, 123.68]))(x)
    
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Initialize model
resnet_model = build_resnet_model(input_shape=(160, 160, 3))
resnet_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 160, 160, 3)       0         
                                                                 
 lambda (Lambda)             (None, 160, 160, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                             

In [6]:
# Assuming you have these from your Xception pipeline
# train, train_labels, val, val_labels

# Add early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
]

# Train the model
history = resnet_model.fit(
    train, train_labels,
    validation_data=(val, val_labels),
    epochs=100,
    batch_size=32,
    callbacks=callbacks
)

Epoch 1/100
417/417 [==============================] - 26s 47ms/step - loss: 0.7205 - accuracy: 0.5721 - val_loss: 0.6421 - val_accuracy: 0.6428
Epoch 2/100
417/417 [==============================] - 18s 44ms/step - loss: 0.6395 - accuracy: 0.6338 - val_loss: 0.6113 - val_accuracy: 0.6759
Epoch 3/100
417/417 [==============================] - 18s 43ms/step - loss: 0.6050 - accuracy: 0.6725 - val_loss: 0.5981 - val_accuracy: 0.6779
Epoch 4/100
417/417 [==============================] - 18s 44ms/step - loss: 0.5814 - accuracy: 0.6956 - val_loss: 0.5787 - val_accuracy: 0.7016
Epoch 5/100
417/417 [==============================] - 18s 44ms/step - loss: 0.5558 - accuracy: 0.7204 - val_loss: 0.5588 - val_accuracy: 0.7157
Epoch 6/100
417/417 [==============================] - 18s 44ms/step - loss: 0.5356 - accuracy: 0.7399 - val_loss: 0.5517 - val_accuracy: 0.7211
Epoch 7/100
417/417 [==============================] - 18s 44ms/step - loss: 0.5122 - accuracy: 0.7494 - val_loss: 0.5291 - val_ac

In [7]:
# Evaluate on test set
test_loss, test_acc = resnet_model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-DF on ResNet50: {test_acc*100:.2f}%')

# Classification report
y_pred = resnet_model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

340/340 [==============================] - 13s 40ms/step - loss: 0.1517 - accuracy: 0.9545

Test Accuracy of Celeb-DF on ResNet50: 95.45%
340/340 [==============================] - 13s 36ms/step
              precision    recall  f1-score   support

           0       0.94      0.89      0.91      2961
           1       0.96      0.98      0.97      7891

    accuracy                           0.95     10852
   macro avg       0.95      0.93      0.94     10852
weighted avg       0.95      0.95      0.95     10852



In [8]:

import tensorflow as tf

# After training the model
resnet_model.save('resnet(50)_160_celeb.h5')  # Saves the entire model to a file


In [9]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 7.1%
RAM Used: 3697.5 MB
Time Usage: 1296.7 s
GPU Memory Used: 1996.4 MB
Power Consumption: 93W


In [5]:
from tensorflow.keras.models import load_model
import tensorflow as tf
print("=== DATA LOADING ===")
start = monitor.get_stats()
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_celeb.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-DF on ResNet50: {test_acc*100:.2f}%')
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===
340/340 [==============================] - 21s 43ms/step - loss: 0.1517 - accuracy: 0.9545

Test Accuracy of Celeb-DF on ResNet50: 95.45%

=== RESOURCE USAGE ===
CPU Usage: 12.3%
RAM Used: -3407.2 MB
Time Usage: 28.6 s
GPU Memory Used: 916.9 MB
Power Consumption: 93W


In [11]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_celeb.h5')
# Evaluate on test set
test_loss, test_acc= model.evaluate(test_hog, test_labels)
print(f'\nTest Accuracy of DFC on Resnet50 from Celebdf: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_hog)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

94/94 [==============================] - 4s 38ms/step - loss: 3.2989 - accuracy: 0.5213

Test Accuracy of DFC on Resnet50 from Celebdf: 52.13%
94/94 [==============================] - 4s 36ms/step
              precision    recall  f1-score   support

           0       0.51      0.86      0.64      1500
           1       0.57      0.18      0.27      1500

    accuracy                           0.52      3000
   macro avg       0.54      0.52      0.46      3000
weighted avg       0.54      0.52      0.46      3000



In [9]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_celeb.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of FF++ on Resnet50 from Celebdf: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_ff)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels_ff, y_pred_binary))

169/169 [==============================] - 7s 37ms/step - loss: 3.3815 - accuracy: 0.3923

Test Accuracy of FF++ on Resnet50 from Celebdf: 39.23%
169/169 [==============================] - 6s 35ms/step
              precision    recall  f1-score   support

           0       0.29      0.83      0.43      1468
           1       0.78      0.23      0.35      3911

    accuracy                           0.39      5379
   macro avg       0.53      0.53      0.39      5379
weighted avg       0.65      0.39      0.37      5379



# DFC

In [2]:
import h5py
import numpy as np
# Open the HDF5 file in read mode
with h5py.File('D://thesis//dataset//deepfake dataset//resized_images.h5', 'r') as h5f:
    # Access each dataset
    celeb = np.array(h5f['celeb'])
    ffhq = np.array(h5f['ffhq'])
    gdwct = np.array(h5f['gdwct'])
    attgan = np.array(h5f['attgan'])
    stargan = np.array(h5f['stargan'])
    stylegan2 = np.array(h5f['stylegan2'])
    stylegan = np.array(h5f['stylegan'])

# Now, 'celeb', 'ffhq', etc., are NumPy arrays containing your datasets
print(f"celeb shape: {celeb.shape}, dtype: {celeb.dtype}")
print(f"ffhq shape: {ffhq.shape}, dtype: {ffhq.dtype}")
print(f"ffhq shape: {gdwct.shape}, dtype: {gdwct.dtype}")
print(f"ffhq shape: {attgan.shape}, dtype: {attgan.dtype}")
print(f"ffhq shape: {stargan.shape}, dtype: {stargan.dtype}")
print(f"ffhq shape: {stylegan2.shape}, dtype: {stylegan2.dtype}")
print(f"ffhq shape: {stylegan.shape}, dtype: {stylegan.dtype}")
# Repeat for other datasets as needed
import cv2
# Function to resize images from (224, 224) to (160, 160)
def resize_images(image_array, target_size=(160, 160)):
    resized_images = np.array([cv2.resize(img, target_size) for img in image_array])
    return resized_images

celeb = resize_images(celeb, target_size=(160, 160))
ffhq = resize_images(ffhq, target_size=(160, 160))
gdwct = resize_images(gdwct, target_size=(160, 160))
attgan = resize_images(attgan, target_size=(160, 160))
stargan = resize_images(stargan, target_size=(160, 160))
stylegan = resize_images(stargan, target_size=(160, 160))
stylegan2 = resize_images(stylegan2, target_size=(160, 160))
import random
# Randomly select 2500 distinct images
random_indices = random.sample(range(len(celeb)), 2500)  # Get 2500 random indices
celeb = celeb[random_indices]  # Select the random subse

import random
# Randomly select 2500 distinct images
random_indices = random.sample(range(len(ffhq)), 2500)  # Get 2500 random indices
ffhq = ffhq[random_indices]  # Select the random subse
print(f"celeb shape: {celeb.shape}, dtype: {celeb.dtype}")
print(f"ffhq shape: {ffhq.shape}, dtype: {ffhq.dtype}")
print(f"gdwct shape: {gdwct.shape}, dtype: {gdwct.dtype}")
print(f"attagan shape: {attgan.shape}, dtype: {attgan.dtype}")
print(f"stargan shape: {stargan.shape}, dtype: {stargan.dtype}")
print(f"stylegan2 shape: {stylegan2.shape}, dtype: {stylegan2.dtype}")
print(f"stylegan shape: {stylegan.shape}, dtype: {stylegan.dtype}")
import random
import numpy as np

def split_data(data, train_ratio=0.7):
    """
    Splits data into training and testing sets based on the specified ratio.

    Parameters:
        data (list or np.array): The dataset to split.
        train_ratio (float): The ratio of the data to include in the training set.

    Returns:
        tuple: Two datasets - train and test.
    """
    # Shuffle the data
    random.shuffle(data)

    # Calculate the split index
    split_index = int(len(data) * train_ratio)

    # Split the data
    train_data = data[:split_index]
    test_data = data[split_index:]

    return train_data, test_data

# Split `celeb` into 70% train and 30% test
celeb_train_hog, celeb_test_hog = split_data(celeb, train_ratio=0.7)

# Split `ffhq` into 70% train and 30% test
ffhq_train_hog, ffhq_test_hog = split_data(ffhq, train_ratio=0.7)

# Split `attgan` into 70% train and 30% test
attgan_train_hog, attgan_test_hog = split_data(attgan, train_ratio=0.7)

# Split `stargan` into 70% train and 30% test
stargan_train_hog, stargan_test_hog = split_data(stargan, train_ratio=0.7)

# Split `gdwct` into 70% train and 30% test
gdwct_train_hog, gdwct_test_hog = split_data(gdwct, train_ratio=0.7)

# Split `stylegan2` into 70% train and 30% test_hog
stylegan2_train_hog, stylegan2_test_hog = split_data(stylegan2, train_ratio=0.7)

# Split `stylegan` into 70% train and 30% test_hog
stylegan_train_hog, stylegan_test_hog = split_data(stylegan, train_ratio=0.7)

# Convert to NumPy arrays if needed
celeb_train_hog, celeb_test_hog = np.array(celeb_train_hog), np.array(celeb_test_hog)
ffhq_train_hog, ffhq_test_hog = np.array(ffhq_train_hog), np.array(ffhq_test_hog)
attgan_train_hog, attgan_test_hog = np.array(attgan_train_hog), np.array(attgan_test_hog)
stargan_train_hog, stargan_test_hog = np.array(stargan_train_hog), np.array(stargan_test_hog)
gdwct_train_hog, gdwct_test_hog = np.array(gdwct_train_hog), np.array(gdwct_test_hog)
stylegan2_train_hog, stylegan2_test_hog = np.array(stylegan2_train_hog), np.array(stylegan2_test_hog)
stylegan_train_hog, stylegan_test_hog = np.array(stylegan_train_hog), np.array(stylegan_test_hog)

# Print results for verification
print(f"celeb_train: {len(celeb_train_hog)} images, celeb_test: {len(celeb_test_hog)} images")
print(f"ffhq_train: {len(ffhq_train_hog)} images, ffhq_test: {len(ffhq_test_hog)} images")
print(f"attgan_train: {len(attgan_train_hog)} images, attgan_test: {len(attgan_test_hog)} images")
print(f"stargan_train: {len(stargan_train_hog)} images, stargan_test: {len(stargan_test_hog)} images")
print(f"gdwct_train: {len(gdwct_train_hog)} images, gdwct_test: {len(gdwct_test_hog)} images")
print(f"stylegan2_train: {len(stylegan2_train_hog)} images, stylegan2_test: {len(stylegan2_test_hog)} images")
print(f"stylegan_train: {len(stylegan_train_hog)} images, stylegan_test: {len(stylegan_test_hog)} images")

########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
celeb_train_hog, celeb_val_hog = extract_validation(celeb_train_hog)
ffhq_train_hog, ffhq_val_hog = extract_validation(ffhq_train_hog)
attgan_train_hog, attgan_val_hog = extract_validation(attgan_train_hog)
stargan_train_hog, stargan_val_hog = extract_validation(stargan_train_hog)
gdwct_train_hog, gdwct_val_hog = extract_validation(gdwct_train_hog)
stylegan2_train_hog, stylegan2_val_hog = extract_validation(stylegan2_train_hog)
stylegan_train_hog, stylegan_val_hog = extract_validation(stylegan_train_hog)

# Print results for verification
print(f"celeb_train: {len(celeb_train_hog)} images, celeb_val: {len(celeb_val_hog)} images")
print(f"ffhq_train: {len(ffhq_train_hog)} images, ffhq_val: {len(ffhq_val_hog)} images")
print(f"attgan_train: {len(attgan_train_hog)} images, attgan_val: {len(attgan_val_hog)} images")
print(f"stargan_train: {len(stargan_train_hog)} images, stargan_val: {len(stargan_val_hog)} images")
print(f"gdwct_train: {len(gdwct_train_hog)} images, gdwct_val: {len(gdwct_val_hog)} images")
print(f"stylegan2_train: {len(stylegan2_train_hog)} images, stylegan2_val: {len(stylegan2_val_hog)} images")
print(f"stylegan_train: {len(stylegan_train_hog)} images, stylegan_val: {len(stylegan_val_hog)} images")
############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


celeb_train_labels = np.zeros(len(celeb_train_hog), dtype=int)
ffhq_train_labels = np.zeros(len(ffhq_train_hog), dtype=int)
atta_train_labels = np.ones(len(attgan_train_hog), dtype=int)
star_train_labels = np.ones(len(stargan_train_hog), dtype=int)
gdwct_train_labels = np.ones(len(gdwct_train_hog), dtype=int)
stylegan2_train_labels = np.ones(len(stylegan2_train_hog), dtype=int)
stylegan_train_labels = np.ones(len(stylegan_train_hog), dtype=int)

# Concatenate all training datasets into a single `train` variable
train_hog = np.concatenate([celeb_train_hog, ffhq_train_hog, attgan_train_hog, stargan_train_hog, gdwct_train_hog, stylegan2_train_hog, stylegan_train_hog], axis=0)
train_labels=np.concatenate([celeb_train_labels, ffhq_train_labels, atta_train_labels, star_train_labels, gdwct_train_labels, stylegan2_train_labels,
                              stylegan_train_labels], axis=0)




celeb_test_labels = np.zeros(len(celeb_test_hog), dtype=int)
ffhq_test_labels = np.zeros(len(ffhq_test_hog), dtype=int)
atta_test_labels = np.ones(len(attgan_test_hog), dtype=int)
star_test_labels = np.ones(len(stargan_test_hog), dtype=int)
gdwct_test_labels = np.ones(len(gdwct_test_hog), dtype=int)
stylegan2_test_labels = np.ones(len(stylegan2_test_hog), dtype=int)
stylegan_test_labels = np.ones(len(stylegan_test_hog), dtype=int)

# Concatenate all testing datasets into a single `test` variable
test_hog = np.concatenate([celeb_test_hog, ffhq_test_hog, attgan_test_hog, stargan_test_hog, gdwct_test_hog, stylegan2_test_hog, stylegan_test_hog], axis=0)
test_labels = np.concatenate([celeb_test_labels, ffhq_test_labels, atta_test_labels, star_test_labels, gdwct_test_labels, stylegan2_test_labels,
                        stylegan_test_labels], axis=0)




celeb_val_labels = np.zeros(len(celeb_val_hog), dtype=int)
ffhq_val_labels = np.zeros(len(ffhq_val_hog), dtype=int)
atta_val_labels = np.ones(len(attgan_val_hog), dtype=int)
star_val_labels = np.ones(len(stargan_val_hog), dtype=int)
gdwct_val_labels = np.ones(len(gdwct_val_hog), dtype=int)
stylegan2_val_labels = np.ones(len(stylegan2_val_hog), dtype=int)
stylegan_val_labels = np.ones(len(stylegan_val_hog), dtype=int)

# Concatenate all validation datasets into a single `val` variable
val_hog = np.concatenate([celeb_val_hog, ffhq_val_hog, attgan_val_hog, stargan_val_hog, gdwct_val_hog, stylegan2_val_hog, stylegan_val_hog], axis=0)
val_labels = np.concatenate([celeb_val_labels, ffhq_val_labels, atta_val_labels, star_val_labels, gdwct_val_labels, stylegan2_val_labels,
                       stylegan_val_labels], axis=0)

# Print the results for verification
print(f"Total train: {len(train_hog)} images")
print(f"Total test: {len(test_hog)} images")
print(f"Total val: {len(val_hog)} images")


# Print results for verification
print(f"Train Labels: {len(train_labels)} ")
print(f"Test Labels: {len(test_labels)} ")
print(f"Val Labels: {len(val_labels)} ")



celeb shape: (5000, 224, 224, 3), dtype: uint8
ffhq shape: (5000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
celeb shape: (2500, 160, 160, 3), dtype: uint8
ffhq shape: (2500, 160, 160, 3), dtype: uint8
gdwct shape: (1000, 160, 160, 3), dtype: uint8
attagan shape: (1000, 160, 160, 3), dtype: uint8
stargan shape: (1000, 160, 160, 3), dtype: uint8
stylegan2 shape: (1000, 160, 160, 3), dtype: uint8
stylegan shape: (1000, 160, 160, 3), dtype: uint8
celeb_train: 1750 images, celeb_test: 750 images
ffhq_train: 1750 images, ffhq_test: 750 images
attgan_train: 700 images, attgan_test: 300 images
stargan_train: 700 images, stargan_test: 300 images
gdwct_train: 700 images, gdwct_test: 300 images
stylegan2_train: 700 images, stylegan2_test: 300 images
stylegan_train: 700 images, stylegan

In [7]:
# Assuming you have these from your Xception pipeline
# train, train_labels, val, val_labels

# Add early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
]

# Train the model
history = resnet_model.fit(
    train_hog, train_labels,
    validation_data=(val_hog, val_labels),
    epochs=100,
    batch_size=32,
    callbacks=callbacks
)

Epoch 1/100
197/197 [==============================] - 16s 49ms/step - loss: 0.4817 - accuracy: 0.7725 - val_loss: 0.2858 - val_accuracy: 0.9014
Epoch 2/100
197/197 [==============================] - 12s 59ms/step - loss: 0.2877 - accuracy: 0.8865 - val_loss: 0.2180 - val_accuracy: 0.9300
Epoch 3/100
197/197 [==============================] - 12s 63ms/step - loss: 0.2167 - accuracy: 0.9205 - val_loss: 0.1853 - val_accuracy: 0.9514
Epoch 4/100
197/197 [==============================] - 11s 54ms/step - loss: 0.1823 - accuracy: 0.9305 - val_loss: 0.1583 - val_accuracy: 0.9557
Epoch 5/100
197/197 [==============================] - 11s 58ms/step - loss: 0.1403 - accuracy: 0.9546 - val_loss: 0.1419 - val_accuracy: 0.9643
Epoch 6/100
197/197 [==============================] - 13s 65ms/step - loss: 0.1218 - accuracy: 0.9592 - val_loss: 0.1214 - val_accuracy: 0.9671
Epoch 7/100
197/197 [==============================] - 13s 64ms/step - loss: 0.0990 - accuracy: 0.9708 - val_loss: 0.1080 - val_ac

In [8]:
# Evaluate on test set
test_loss, test_acc = resnet_model.evaluate(test_hog, test_labels)
print(f'\nTest Accuracy of DFC on ResNet50: {test_acc*100:.2f}%')

# Classification report
y_pred = resnet_model.predict(test_hog)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

94/94 [==============================] - 4s 39ms/step - loss: 0.1292 - accuracy: 0.9610

Test Accuracy of DFC on ResNet50: 96.10%
94/94 [==============================] - 4s 36ms/step
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1500
           1       0.96      0.96      0.96      1500

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000



In [10]:
import tensorflow as tf
# After training the model
resnet_model.save('resnet(50)_160_dfc.h5')  # Saves the entire model to a file

In [11]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 12.6%
RAM Used: 4294.0 MB
Time Usage: 569.5 s
GPU Memory Used: 117.6 MB
Power Consumption: 93W


In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
print("=== DATA LOADING ===")
start = monitor.get_stats()
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_dfc.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_hog, test_labels)
print(f'\nTest Accuracy of Celeb-DF on ResNet50: {test_acc*100:.2f}%')
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===
94/94 [==============================] - 8s 42ms/step - loss: 0.1416 - accuracy: 0.9480

Test Accuracy of Celeb-DF on ResNet50: 94.80%

=== RESOURCE USAGE ===
CPU Usage: 3.5%
RAM Used: 4052.1 MB
Time Usage: 13.2 s
GPU Memory Used: 341.8 MB
Power Consumption: 93W


In [12]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_dfc.h5')
test_loss, test_acc = model.evaluate(test_hog, test_labels)
print(f'Test Accuracy of DFC on RESNET: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference


94/94 [==============================] - 4s 40ms/step - loss: 0.1292 - accuracy: 0.9610
Test Accuracy of DFC on RESNET: 96.10%


In [6]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_dfc.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-df dataset on Resnet50 for DFC: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))


340/340 [==============================] - 14s 39ms/step - loss: 1.3625 - accuracy: 0.6456

Test Accuracy of Celeb-df dataset on Resnet50 for DFC: 64.56%
340/340 [==============================] - 13s 37ms/step
              precision    recall  f1-score   support

           0       0.29      0.20      0.24      2961
           1       0.73      0.81      0.77      7891

    accuracy                           0.65     10852
   macro avg       0.51      0.51      0.50     10852
weighted avg       0.61      0.65      0.62     10852



In [8]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_dfc.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of FF++ on Resnet50t from DFC: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_ff)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels_ff, y_pred_binary))

169/169 [==============================] - 7s 41ms/step - loss: 1.7480 - accuracy: 0.5635

Test Accuracy of FF++ on Resnet50t from DFC: 56.35%
169/169 [==============================] - 7s 38ms/step
              precision    recall  f1-score   support

           0       0.32      0.53      0.40      1468
           1       0.77      0.58      0.66      3911

    accuracy                           0.56      5379
   macro avg       0.54      0.55      0.53      5379
weighted avg       0.64      0.56      0.59      5379



# FF++

In [2]:
import h5py

# Open the HDF5 file for reading
with h5py.File("D://thesis//dataset//ff++//images_ff++_orignal_processed.h5", "r") as h5f:
    org_seq_actor_array = h5f["ori_actor"][:]
    org_seq_youtube_array = h5f["ori_youtube"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake1_processed.h5", "r") as h5f:
    meni_seq_dfd_array = h5f["mni_dfd"][:]
    meni_seq_df_array = h5f["mni_df"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake2_processed.h5", "r") as h5f:
    meni_seq_f2f_array = h5f["mni_f2f"][:]
    meni_seq_fshifter_array = h5f["mni_fshifter"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake3_processed.h5", "r") as h5f:
    meni_seq_fswap_array = h5f["mni_fswap"][:]
    meni_seq_nt_array = h5f["mni_nt"][:]
print("Data loaded successfully from the HDF5 file.")
# Output the shape of the resulting array
print(f"Shape of the concatenated array: {org_seq_actor_array.shape}")
print(f"Shape of the concatenated array: {org_seq_youtube_array.shape}")

print(f"Shape of the concatenated array: {meni_seq_dfd_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_df_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_fshifter_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_f2f_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_fswap_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_nt_array.shape}")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

ori_actor_train_real, ori_actor_test_real = train_test_split(org_seq_actor_array, test_size=0.3, random_state=42)
ori_youtube_train_real, ori_youtube_test_real = train_test_split(org_seq_youtube_array, test_size=0.3, random_state=42)
# Split the data into train (70%) and test (30%)
mni_dfd_train_fake, mni_dfd_test_fake = train_test_split(meni_seq_dfd_array, test_size=0.3, random_state=42)
mni_df_train_fake, mni_df_test_fake = train_test_split(meni_seq_df_array, test_size=0.3, random_state=42)
mni_f2f_train_fake, mni_f2f_test_fake = train_test_split(meni_seq_f2f_array, test_size=0.3, random_state=42)
mni_fshifter_train_fake, mni_fshifter_test_fake = train_test_split(meni_seq_fshifter_array, test_size=0.3, random_state=42)
mni_fswap_train_fake, mni_fswap_test_fake = train_test_split(meni_seq_fswap_array, test_size=0.3, random_state=42)
mni_nt_train_fake, mni_nt_test_fake = train_test_split(meni_seq_nt_array, test_size=0.3, random_state=42)


# Output the shape to confirm the split dimensions
print("Training data ACTOR real shape:", ori_actor_train_real.shape, "Testing real data shape:", ori_actor_test_real.shape)
print("Training data Youtube real shape:", ori_youtube_train_real.shape, "Testing real data shape:", ori_youtube_test_real.shape)

print("Training data DFD fake shape:", mni_dfd_train_fake.shape, "Testing fake data shape:", mni_dfd_test_fake.shape)
print("Training data DF fake shape:", mni_df_train_fake.shape, "Testing fake data shape:", mni_df_test_fake.shape)
print("Training data f2f fake shape:", mni_f2f_train_fake.shape, "Testing fake data shape:", mni_f2f_test_fake.shape)
print("Training data fshifter fake shape:", mni_fshifter_train_fake.shape, "Testing fake data shape:", mni_fshifter_test_fake.shape)
print("Training data fswap fake shape:", mni_fswap_train_fake.shape, "Testing fake data shape:", mni_fswap_test_fake.shape)
print("Training data nt fake shape:", mni_nt_train_fake.shape, "Testing fake data shape:", mni_nt_test_fake.shape)
########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
train_hog_ori_actor_real, val_hog_ori_actor_real = extract_validation(ori_actor_train_real)
train_hog_ori_youtube_real, val_hog_ori_youtube_real = extract_validation(ori_youtube_train_real)
train_hog_mni_dfd_fake, val_hog_mni_dfd_fake = extract_validation(mni_dfd_train_fake)
train_hog_mni_df_fake, val_hog_mni_df_fake = extract_validation(mni_df_train_fake)
train_hog_mni_f2f_fake, val_hog_mni_f2f_fake = extract_validation(mni_f2f_train_fake)
train_hog_mni_fshifter_fake, val_hog_mni_fshifter_fake = extract_validation(mni_fshifter_train_fake)
train_hog_mni_fswap_fake, val_hog_mni_fswap_fake = extract_validation(mni_fswap_train_fake)
train_hog_mni_nt_fake, val_hog_mni_nt_fake = extract_validation(mni_nt_train_fake)
# Print results for verification
print(f"train_ori_actor hog_real: {len(train_hog_ori_actor_real)} images, val_ori_actor hog_real: {len(val_hog_ori_actor_real)} images")
print(f"train_ ori_youtube hog_real: {len(train_hog_ori_youtube_real)} images, val_ ori_youtube hog_real: {len(val_hog_ori_youtube_real)} images")
print(f"train_hog_mni_dfd_fake: {len(train_hog_mni_dfd_fake)} images, val_hog_mni_dfd_fake: {len(val_hog_mni_dfd_fake)} images")
print(f"train_hog_mni_df_fake: {len(train_hog_mni_df_fake)} images, val_hog_mni_df_fake: {len(val_hog_mni_df_fake)} images")
print(f"train_hog_mni_f2f_fake: {len(train_hog_mni_f2f_fake)} images, val_hog_mni_f2f_fake: {len(val_hog_mni_f2f_fake)} images")
print(f"train_hog_mni_fshifter_fake: {len(train_hog_mni_fshifter_fake)} images, val_hog_mni_fshifter_fake: {len(val_hog_mni_fshifter_fake)} images")
print(f"train_hog_mni_fswap_fake: {len(train_hog_mni_fswap_fake)} images, val_hog_mni_fswap_fake: {len(val_hog_mni_fswap_fake)} images")
print(f"train_hog_mni_nt_fake: {len(train_hog_mni_nt_fake)} images, val_hog_mni_nt_fake: {len(val_hog_mni_nt_fake)} images")
############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


train_ori_actor_labels_real = np.zeros(len(train_hog_ori_actor_real), dtype=int)
train_ori_youtube_labels_real = np.zeros(len(train_hog_ori_youtube_real), dtype=int)
train_mni_dfd_labels_fake = np.ones(len(train_hog_mni_dfd_fake), dtype=int)
train_mni_df_labels_fake = np.ones(len(train_hog_mni_df_fake), dtype=int)
train_mni_f2f_labels_fake = np.ones(len(train_hog_mni_f2f_fake), dtype=int)
train_mni_fshifter_labels_fake = np.ones(len(train_hog_mni_fshifter_fake), dtype=int)
train_mni_fswap_labels_fake = np.ones(len(train_hog_mni_fswap_fake), dtype=int)
train_mni_nt_labels_fake = np.ones(len(train_hog_mni_nt_fake), dtype=int)

test_ori_actor_labels_real = np.zeros(len(ori_actor_test_real), dtype=int)
test_ori_youtube_labels_real = np.zeros(len(ori_youtube_test_real), dtype=int)
test_mni_dfd_labels_fake = np.ones(len(mni_dfd_test_fake), dtype=int)
test_mni_df_labels_fake = np.ones(len(mni_df_test_fake), dtype=int)
test_mni_f2f_labels_fake = np.ones(len(mni_f2f_test_fake), dtype=int)
test_mni_fshifter_labels_fake = np.ones(len(mni_fshifter_test_fake), dtype=int)
test_mni_fswap_labels_fake = np.ones(len(mni_fswap_test_fake), dtype=int)
test_mni_nt_labels_fake = np.ones(len(mni_nt_test_fake), dtype=int)


val_ori_actor_labels_real = np.zeros(len(val_hog_ori_actor_real), dtype=int)
val_ori_youtube_labels_real = np.zeros(len(val_hog_ori_youtube_real), dtype=int)
val_mni_dfd_labels_fake = np.ones(len(val_hog_mni_dfd_fake), dtype=int)
val_mni_df_labels_fake = np.ones(len(val_hog_mni_df_fake), dtype=int)
val_mni_f2f_labels_fake = np.ones(len(val_hog_mni_f2f_fake), dtype=int)
val_mni_fshifter_labels_fake = np.ones(len(val_hog_mni_fshifter_fake), dtype=int)
val_mni_fswap_labels_fake = np.ones(len(val_hog_mni_fswap_fake), dtype=int)
val_mni_nt_labels_fake = np.ones(len(val_hog_mni_nt_fake), dtype=int)
##################################################################################################################

# Concatenate all training datasets into a single `train` variable
train_ff = np.concatenate([train_hog_ori_actor_real, train_hog_ori_youtube_real,train_hog_mni_dfd_fake,train_hog_mni_df_fake,
                            train_hog_mni_f2f_fake,train_hog_mni_fshifter_fake,train_hog_mni_fswap_fake,train_hog_mni_nt_fake], axis=0)

train_labels_ff = np.concatenate([train_ori_actor_labels_real, train_ori_youtube_labels_real,train_mni_dfd_labels_fake,train_mni_df_labels_fake,
                            train_mni_f2f_labels_fake,train_mni_fshifter_labels_fake,train_mni_fswap_labels_fake,train_mni_nt_labels_fake], axis=0)

# Concatenate all validation datasets into a single `val` variable
val_ff = np.concatenate([val_hog_ori_actor_real, val_hog_ori_youtube_real, val_hog_mni_dfd_fake, val_hog_mni_df_fake,
                            val_hog_mni_f2f_fake, val_hog_mni_fshifter_fake, val_hog_mni_fswap_fake, val_hog_mni_nt_fake], axis=0)
val_labels_ff = np.concatenate([val_ori_actor_labels_real, val_ori_youtube_labels_real, val_mni_dfd_labels_fake, val_mni_df_labels_fake,
                            val_mni_f2f_labels_fake, val_mni_fshifter_labels_fake, val_mni_fswap_labels_fake, val_mni_nt_labels_fake], axis=0)
# Concatenate all testing datasets into a single `test` variable
test_ff = np.concatenate([ori_actor_test_real, ori_youtube_test_real, mni_dfd_test_fake,
                           mni_df_test_fake, mni_f2f_test_fake, mni_fshifter_test_fake,
                           mni_fswap_test_fake, mni_nt_test_fake], axis=0)
test_labels_ff = np.concatenate([test_ori_actor_labels_real, test_ori_youtube_labels_real, test_mni_dfd_labels_fake, test_mni_df_labels_fake,
                            test_mni_f2f_labels_fake, test_mni_fshifter_labels_fake, test_mni_fswap_labels_fake, test_mni_nt_labels_fake], axis=0)


# Print the results for verification
# Print the results for verification
print(f"Total train: {len(train_ff)} images, and shape:{train_ff.shape}")
print(f"Total test: {len(test_ff)} images, and shape:{test_ff.shape}")
print(f"Total val: {len(val_ff)} images, and shape:{val_ff.shape}")


# Print results for verification
print(f"Train Labels: {len(train_labels_ff)} ")
print(f"Test Labels: {len(test_labels_ff)} ")
print(f"Val Labels: {len(val_labels_ff)} ")





Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Shape of the concatenated array: (2808, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (3299, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (1742, 160, 160, 3)
Shape of the concatenated array: (1742, 160, 160, 3)
Training data ACTOR real shape: (1965, 160, 160, 3) Testing real data shape: (843, 160, 160, 3)
Training data Youtube real shape: (1458, 160, 160, 3) Testing real data shape: (625, 160, 160, 3)
Training data DFD fake shape: (2309, 160, 160, 3) Testing fake data shape: (990, 160, 160, 3)
Training data DF fake shape: (1458, 160, 160, 3) Testing fake data shape: (625, 160, 160, 3)
Training data f

In [12]:
# Assuming you have these from your Xception pipeline
# train, train_labels, val, val_labels



# Train the model
history = resnet_model.fit(
    train_ff, train_labels_ff,
    validation_data=(val_ff, val_labels_ff),
    epochs=100,
    batch_size=32
)

Epoch 1/100
353/353 [==============================] - 15s 43ms/step - loss: 0.4788 - accuracy: 0.7893 - val_loss: 0.5273 - val_accuracy: 0.7651
Epoch 2/100
353/353 [==============================] - 15s 44ms/step - loss: 0.4739 - accuracy: 0.7942 - val_loss: 0.5308 - val_accuracy: 0.7659
Epoch 3/100
353/353 [==============================] - 15s 43ms/step - loss: 0.4701 - accuracy: 0.7935 - val_loss: 0.5257 - val_accuracy: 0.7715
Epoch 4/100
353/353 [==============================] - 15s 43ms/step - loss: 0.4639 - accuracy: 0.7995 - val_loss: 0.5235 - val_accuracy: 0.7651
Epoch 5/100
353/353 [==============================] - 15s 44ms/step - loss: 0.4573 - accuracy: 0.8029 - val_loss: 0.5306 - val_accuracy: 0.7611
Epoch 6/100
353/353 [==============================] - 15s 44ms/step - loss: 0.4526 - accuracy: 0.8039 - val_loss: 0.5309 - val_accuracy: 0.7723
Epoch 7/100
353/353 [==============================] - 15s 44ms/step - loss: 0.4464 - accuracy: 0.8078 - val_loss: 0.5293 - val_ac

In [13]:
# Evaluate on test set
test_loss, test_acc = resnet_model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of FF++ on ResNet50: {test_acc*100:.2f}%')

# Classification report
y_pred = resnet_model.predict(test_ff)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels_ff, y_pred_binary))

169/169 [==============================] - 6s 38ms/step - loss: 0.8502 - accuracy: 0.7620

Test Accuracy of FF++ on ResNet50: 76.20%
169/169 [==============================] - 6s 36ms/step
              precision    recall  f1-score   support

           0       0.61      0.37      0.46      1468
           1       0.79      0.91      0.85      3911

    accuracy                           0.76      5379
   macro avg       0.70      0.64      0.65      5379
weighted avg       0.74      0.76      0.74      5379



In [15]:
import tensorflow as tf
# After training the model
resnet_model.save('resnet(50)_160_ff.h5')  # Saves the entire model to a file

In [16]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 11.1%
RAM Used: -564.3 MB
Time Usage: 1863.4 s
GPU Memory Used: 1863.2 MB
Power Consumption: 93W


In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_ff.h5')
test_loss, test_acc = model.evaluate(test_hog, test_labels)
print(f'Test Accuracy of DFC on RESNET from FF++: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference


169/169 [==============================] - 7s 38ms/step - loss: 0.8502 - accuracy: 0.7620
Test Accuracy of DFC on RESNET: 76.20%


In [12]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_ff.h5')
test_loss, test_acc = model.evaluate(test_hog, test_labels)
print(f'Test Accuracy of DFC on Resnet50 from FF++: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference


94/94 [==============================] - 5s 39ms/step - loss: 2.7653 - accuracy: 0.5313
Test Accuracy of DFC on Resnet50 from FF++: 53.13%


In [5]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_ff.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-df dataset on Inceptionnet for FF++: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))


340/340 [==============================] - 23s 41ms/step - loss: 1.2038 - accuracy: 0.7033

Test Accuracy of Celeb-df dataset on Inceptionnet for FF++: 70.33%
340/340 [==============================] - 13s 37ms/step
              precision    recall  f1-score   support

           0       0.34      0.10      0.15      2961
           1       0.73      0.93      0.82      7891

    accuracy                           0.70     10852
   macro avg       0.54      0.51      0.49     10852
weighted avg       0.63      0.70      0.64     10852



In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
print("=== DATA LOADING ===")
start = monitor.get_stats()
# Load the saved model
model = tf.keras.models.load_model('resnet(50)_160_ff.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of Celeb-DF on ResNet50: {test_acc*100:.2f}%')
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===
169/169 [==============================] - 12s 41ms/step - loss: 0.8502 - accuracy: 0.7622

Test Accuracy of Celeb-DF on ResNet50: 76.22%

=== RESOURCE USAGE ===
CPU Usage: 4.7%
RAM Used: 4094.4 MB
Time Usage: 17.6 s
GPU Memory Used: 516.0 MB
Power Consumption: 93W
